## Exploratory Data Analysis

In [22]:
import pandas as pd
from copy import deepcopy
import nltk

In [2]:
df = pd.read_csv("amazon_movie_reviews.csv")
df.head()

/var/folders/3w/mwx278b95z5b8wh63zkvl7xm0000gn/T/ipykernel_6054/2540728358.py:1: DtypeWarning: Columns (18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("amazon_movie_reviews.csv")


,Unnamed: 0,rating,review_title,text,images_x,asin,parent_asin,user_id,timestamp,helpful_vote,...,features,description,price,images_y,videos,store,categories,details,bought_together,author
0,0,5.0,Five Stars,"Amazon, please buy the show! I'm hooked!",[],B013488XFS,B013488XFS,AGGZ357AO26RQZVRLGU4D4N52DZQ,1440385637000,0,...,"['IMDb 8.1', '2017', '10 episodes', 'X-Ray', '...",['A\xa0con man (Giovanni Ribisi) on the run fr...,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,"['Suspense', 'Drama']","{'Content advisory': ['Nudity', 'violence', 's...",NaN,NaN
1,1,5.0,Five Stars,My Kiddos LOVE this show!!,[],B00CB6VTDS,B00CB6VTDS,AGKASBHYZPGTEPO6LWZPVJWB2BVA,1461100610000,0,...,"['2014', '13 episodes', 'X-Ray', 'ALL']",['Follow the adventures of Arty and his sideki...,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,['Kids'],{'Audio languages': ['English Dialogue Boost: ...,NaN,NaN
2,2,3.0,Some decent moments...but...,Annabella Sciorra did her character justice wi...,[],B096Z8Z3R6,B096Z8Z3R6,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,1646271834582,0,...,NaN,NaN,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,NaN,"{'Content advisory': ['Violence', 'substance u...",NaN,NaN
3,3,4.0,"Decent Depiction of Lower-Functioning Autism, ...",...there should be more of a range of characte...,[],B09M14D9FZ,B09M14D9FZ,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,1645937761864,1,...,NaN,NaN,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,NaN,"{'Content advisory': ['Violence', 'alcohol use...",NaN,NaN
4,4,5.0,What Love Is...,"...isn't always how you expect it to be, but w...",[],B001H1SVZC,B001H1SVZC,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,1590639227074,0,...,NaN,NaN,NaN,[{'360w': 'https://images-na.ssl-images-amazon...,[],NaN,NaN,"{'Subtitles': ['None available'], 'Directors':...",NaN,NaN


In [28]:
len(df)

1000000

In [29]:
# Only extracting features relevant to reviews

df_processed = deepcopy(df[["rating", "review_title", "text", "helpful_vote", "average_rating"]])
# does price matter?                   

In [30]:
df_processed.head()

,rating,review_title,text,helpful_vote,average_rating
0,5.0,Five Stars,"Amazon, please buy the show! I'm hooked!",0,4.6
1,5.0,Five Stars,My Kiddos LOVE this show!!,0,4.8
2,3.0,Some decent moments...but...,Annabella Sciorra did her character justice wi...,0,3.9
3,4.0,"Decent Depiction of Lower-Functioning Autism, ...",...there should be more of a range of characte...,1,4.8
4,5.0,What Love Is...,"...isn't always how you expect it to be, but w...",0,4.5


## *Testing models/params etc*

In [31]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/punitaverma/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [53]:
type(df_processed.iloc[0,2])
# x = encode(pd.Series(df_processed.iloc[0,2]), 'Bag-of-Words')

['amazon' 'buy' 'hooked' 'please' 'show' 'the']


In [55]:
sid.polarity_scores(df_processed.iloc[0,2])

{'neg': 0.0, 'neu': 0.522, 'pos': 0.478, 'compound': 0.555}

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
# import pandas as pd

# Download the NLTK tokenizer models (only the first time)
nltk.download('punkt')

def preprocess(corpus):
    tokenized_corpus = []
    for i, word in corpus.items():
        tokenized_corpus.append(nltk.word_tokenize(word))
    print(tokenized_corpus)
    print([word for doc in tokenized_corpus for word in doc])
    vocab = set()
    for doc in tokenized_corpus:
        for word in doc:
            vocab.update(word)
    return tokenized_corpus, vocab

def encode(corpus, encoding_method):
    if encoding_method == 'Bag-of-Words':
        bow_vectorizer = CountVectorizer()
        bow_matrix = bow_vectorizer.fit_transform(corpus)
        print(bow_vectorizer.get_feature_names_out())
        return bow_matrix.toarray()
    
    elif encoding_method == 'TF-IDF':
        tdif_vectorizer = TfidfVectorizer()
        tfidf_matrix = tdif_vectorizer.fit_transform(corpus)
        print(tdif_vectorizer.get_feature_names_out())
        return tfidf_matrix.toarray()
    
    elif encoding_method == 'Word2Vec':
        model = Word2Vec([word_tokenize(doc) for doc in corpus], vector_size=20, window=5, min_count=1, workers=4)
        #word_vectors = {word: model.wv[word] for word in model.wv.vocab}
        return model


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/punitaverma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [63]:
df_processed.dropna(subset=['text'], inplace=True)

In [67]:
type(df_processed['text'][0])

str

In [69]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

# Create an instance of the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Convert non-string values to strings in the 'text' column
df['text'] = df['text'].astype(str)

# Analyze sentiment for each review text
df['sentiment'] = df['text'].apply(lambda x: sid.polarity_scores(x)['compound'])

# Convert sentiment scores to sentiment labels (positive, neutral, negative)
df['sentiment_label'] = pd.cut(df['sentiment'], bins=[-1, -0.5, 0.5, 1], labels=['negative', 'neutral', 'positive'])

# Print the first few rows of the DataFrame with sentiment analysis results
print(df[['text', 'sentiment', 'sentiment_label']].head())

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/punitaverma/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


                                                text  sentiment  \
0           Amazon, please buy the show! I'm hooked!     0.5550   
1                         My Kiddos LOVE this show!!     0.7592   
2  Annabella Sciorra did her character justice wi...     0.7657   
3  ...there should be more of a range of characte...     0.9054   
4  ...isn't always how you expect it to be, but w...     0.8957   

  sentiment_label  
0        positive  
1        positive  
2        positive  
3        positive  
4        positive  
